In [2]:
#!pip install selenium
#!pip install webdriver_manager

In [2]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
# start timer
start_time = time.time()

# according download: https://googlechromelabs.github.io/chrome-for-testing/#stable | caution: chromeversion and chromewbdriver version have to match, to check chromeversion --> paste chrome://version/ into chrome
options = Options()
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
chrome_driver_path = r'C:\Program Files\WebDriver\chromedriver-win64\chromedriver.exe'
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)

all_prices = []
all_car_names = []
all_locations = []
all_infos = [] 

def slow_scroll(driver, scroll_pause_time=0.05, total_scroll_time=1):
    # Get the current scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Removed the stray word and corrected indentation below
    scroll_start_time = time.time()  # Use a different variable name to avoid conflict
    while True:
        # Calculate the current time elapsed
        current_time = time.time()
        if current_time - scroll_start_time >= total_scroll_time:
            break  # Stop scrolling after the specified total_scroll_time

        # Scroll down by a small amount
        driver.execute_script("window.scrollBy(0, 500);")  # Adjust the number of pixels per scroll if needed
        
        # Wait a short amount of time
        time.sleep(scroll_pause_time)

for page in range(0, 5):  # Loop
    url = f'https://www.autoscout24.ch/de/s?pagination%5Bpage%5D={page}'
    driver.get(url)
    
    if page == 0:
        cookie_accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
        )
        cookie_accept_button.click()
       
    # Slowly scroll through the website over 5 seconds
    slow_scroll(driver, 0.1, 2)

    price_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'p.chakra-text.css-bwl0or'))
    )
    prices = [element.text for element in price_elements]
    all_prices.extend(prices)

    name_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'h2.chakra-heading.css-svmyrf'))
    )
    car_names = [element.text for element in name_elements]
    all_car_names.extend(car_names)

    location_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'p.chakra-text.css-fkcxb5'))
    )
    locations = [element.text for element in location_elements]
    all_locations.extend(locations)

    info_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.css-1kcev50 p.chakra-text.css-rlwdn6'))
    )
    infos = [element.text for element in info_elements]
    all_infos.extend(infos)

    dates = all_infos[0::6]
    km = all_infos[1::6]
    transmission = all_infos[2::6]
    fuel = all_infos[3::6]
    ps = all_infos[4::6]
    consumption = all_infos[5::6]

    df = pd.DataFrame({
        'Car': all_car_names, 
        'Price': all_prices, 
        'Date': dates, 
        'Km': km, 
        'Transmission': transmission, 
        'Fuel': fuel, 
        'PS': ps, 
        'Consumption': consumption,
        'Location': all_locations
    })

driver.quit()

#end timer
end_time = time.time()

# Calculate time needed
elapsed_time = end_time - start_time

KeyboardInterrupt: 

In [29]:
print(f"Total execution time: {(elapsed_time)} seconds")

Total execution time: 28.38508915901184 seconds


In [27]:
df.head()

,Car,Price,Dates,KM,Transmission,Fuel,PS,Consumption,Locations
0,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,05.2012,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
1,AUDI RS e-tron GT quattro,CHF 155'990.–,Neues Fahrzeug,9 km,Automatikgetriebe,Elektro,599 PS (440 kW),469 km,4147 Aesch
2,BMW 120d xDrive Sport Line,CHF 36'900.–,05.2023,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,03.2016,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
4,MERCEDES-BENZ B 250 e AMG Line 8G-DCT,CHF 37'900.–,11.2020,9'500 km,Halbautomatisches Getriebe,Plug-in hybrid Benzin/Elektro,217 PS (160 kW),1.5 l/100 km,6438 Ibach-Schwyz
